- [ ] Ler todos os arquivos
- [ ] Separar por municipios
- [ ] Transformacoes
   - Limpeza
   - Atributos
- [ ] Salvar no banco

In [7]:
import pandas as pd

df = pd.read_parquet('1.limpo.parquet').astype({'data': 'datetime64[ns]'})
len(df)

10298930

In [8]:
def remover_colunas(df):
  df.drop(columns=["Recuperadosnovos", "emAcompanhamentoNovos"], inplace=True)
  return df

In [9]:
def obter_estacao(data):
    ano = data.year
    datas_estacoes = {
        "verao": pd.Timestamp(f"{ano}-12-21"),
        "outono": pd.Timestamp(f"{ano}-3-21"),
        "inverno": pd.Timestamp(f"{ano}-6-21"),
        "primavera": pd.Timestamp(f"{ano}-9-21"),
    }

    if data >= datas_estacoes["verao"] or data < datas_estacoes["outono"]:
        return "Verão"
    elif data >= datas_estacoes["outono"] and data < datas_estacoes["inverno"]:
        return "Outono"
    elif data >= datas_estacoes["inverno"] and data < datas_estacoes["primavera"]:
        return "Inverno"
    else:
        return "Primavera"

def traduzir_dias_da_semana(ingles):
  portugues = {
     "Sunday": "Domingo",
     "Monday": "Segunda-feira",
     "Tuesday": "Terça-feira",
     "Wednesday": "Quarta-feira",
     "Thursday": "Quinta-feira",
     "Friday": "Sexta-feira",
     "Saturday": "Sábado"
  }

  return portugues[ingles]

def traduzir_mes(ingles):
  portuguese = {
      "January": "Janeiro",
      "February": "Fevereiro",
      "March": "Março",
      "April": "Abril",
      "May": "Maio",
      "June": "Junho",
      "July": "Julho",
      "August": "Agosto",
      "September": "Setembro",
      "October": "Outubro",
      "November": "Novembro",
      "December": "Dezembro"
  }

  return portuguese[ingles]

def adicionar_feature_datas(df):
  df["mes"] = df.data.dt.month_name()
  df["mes_traduzido"] = df["mes"].apply(traduzir_mes)
  df["dia_semana"] = df.data.dt.day_name()
  df["dia_semana_traduzido"] = df["dia_semana"].apply(traduzir_dias_da_semana)
  df["estacao"] = df["data"].apply(obter_estacao)

  return df

In [10]:
def rodar_engenharia_de_atributos(df):
  df = remover_colunas(df)
  df = adicionar_feature_datas(df)

  return df


rodar_engenharia_de_atributos(df)
len(df)

10298930

In [11]:
df.to_parquet('2.atributos.parquet', index=False)

In [12]:
df = pd.read_parquet('2.atributos.parquet')

df.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,...,casosNovos,obitosAcumulado,obitosNovos,interior/metropolitana,novos_casos_acumulados,mes,mes_traduzido,dia_semana,dia_semana_traduzido,estacao
0,Norte,AC,Acrelândia,12,120001.0,12002.0,BAIXO ACRE E PURUS,2020-03-27,13,15256.0,...,0,0,0,0.0,0,March,Março,Friday,Sexta-feira,Outono
1,Norte,AC,Acrelândia,12,120001.0,12002.0,BAIXO ACRE E PURUS,2020-03-28,13,15256.0,...,0,0,0,0.0,0,March,Março,Saturday,Sábado,Outono
2,Norte,AC,Acrelândia,12,120001.0,12002.0,BAIXO ACRE E PURUS,2020-03-29,14,15256.0,...,2,0,0,0.0,2,March,Março,Sunday,Domingo,Outono
3,Norte,AC,Acrelândia,12,120001.0,12002.0,BAIXO ACRE E PURUS,2020-03-30,14,15256.0,...,4,0,0,0.0,6,March,Março,Monday,Segunda-feira,Outono
4,Norte,AC,Acrelândia,12,120001.0,12002.0,BAIXO ACRE E PURUS,2020-03-31,14,15256.0,...,1,0,0,0.0,7,March,Março,Tuesday,Terça-feira,Outono
